In [54]:
"""
Notebook for post-processing the results
"""
import os
import numpy as np
import pandas as pd

database = "./database/"
filename = "input-uci-crispr-experiments.csv"

# Read Data
data = pd.read_csv(os.path.join(database,filename))
## Sort out the job that has been finished
finished_data = data[data['State'] == 'finished']

def get_mean_std(dt):
    """Calculate the Mean and Std of the results"""
    s = []
    for col in dt.columns:
        s.append((np.mean(dt[col]), np.std(dt[col])))
    return s

image = ['mnist', 'fmnist', 'cifar10']
ls = []

for dataset in image:
    result = {}
    data_class = finished_data[(finished_data['dataset'] == dataset)]

    # homoscedastic
    homo_data = data_class[(data_class['likelihood'] == 'homoscedastic') & (data_class['method'] == 'marglik')]
    homo_eb_pp = homo_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['homo'] = get_mean_std(homo_eb_pp)

    # naive
    naive_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'map')]
    naive_gs = naive_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['naive_gs'] = get_mean_std(naive_gs)

    # Other methods
    betahalf_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'betanll') & (data_class['beta'] == 0.5)]
    betahalf_gs = betahalf_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['betahalf'] = get_mean_std(betahalf_gs)

    betaone_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'betanll') & (data_class['beta'] == 1)]
    betaone_gs = betaone_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['betaone'] = get_mean_std(betaone_data[['test/loglik', 'test/kl_div', 'test/rmse']])

    faith_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'faithful')]
    faith_gs = faith_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['faith'] = get_mean_std(faith_gs)
    
    mcdropout_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'mcdropout')]
    mcdropout_gs_pp = mcdropout_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['mcdropout'] = get_mean_std(mcdropout_gs_pp)

    vi_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'vi')]
    vi_gs_pp = vi_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['vi'] = get_mean_std(vi_gs_pp)

    # naive
    naive_eb_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'marglik')]
    naive_eb_pp = naive_eb_pp_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['naive_eb_pp'] = get_mean_std(naive_eb_pp)

    naive_eb_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'marglik')]
    naive_eb = naive_eb_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['naive_eb'] = get_mean_std(naive_eb)

    # natural
    natural_gs_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'map')]
    natural_gs_pp = natural_gs_pp_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['natural_gs_pp'] = get_mean_std(natural_gs_pp)

    natural_gs_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'map')]
    natural_gs = natural_gs_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['natural_gs'] = get_mean_std(natural_gs)

    natural_eb_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'marglik')]
    natural_eb_pp = natural_eb_pp_data[['test/loglik_bayes', 'test/kl_div', 'test/rmse']]
    result['natural_eb_pp'] = get_mean_std(natural_eb_pp)

    natural_eb_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'marglik')]
    natural_eb = natural_eb_data[['test/loglik', 'test/kl_div', 'test/rmse']]
    result['natural_eb'] = get_mean_std(natural_eb)

    ls.append(result)

    # Define the columns of the table
columns = pd.MultiIndex.from_product(
    [['MNIST with MLP', 'FashionMNIST with CNN', 'CIFAR10'], ['LL', 'D_KL', 'RMSE']],
    names=['Dataset', 'Metric']
)

# define the rows of the table
methods = [
    ('Homoscedastic', 'EB', '✓'),
    ('Naive NLL', 'GS', '✗'),
    ('β-NLL (0.5)', 'GS', '✗'),
    ('β-NLL (1)', 'GS', '✗'),
    ('Faithful', 'GS', '✗'),
    ('MC-Dropout', 'GS', '✓'),
    ('VI', 'GS', '✓'),
    ('Naive NLL', 'EB', '✓'),
    ('Naive NLL', 'EB', '✗'),
    ('Natural NLL', 'GS', '✓'),
    ('Natural NLL', 'GS', '✗'),
    ('Natural NLL', 'EB', '✓'),
    ('Natural NLL', 'EB', '✗')
]

# Define the methods to be used in the table
table_data_method = [
    'homo',
    'naive_gs',
    'betahalf',
    'betaone',
    'faith',
    'mcdropout',
    'vi',
    'naive_eb_pp',
    'naive_eb_pp',
    'natural_gs_pp',
    'natural_gs',
    'natural_eb_pp',
    'natural_eb'
]


table_data = []

# Fill the table
for method_name in table_data_method:
    row = []
    for dataset in ls:  # mnist, fmnist, cifar10
        method_key = method_name
        if method_key in dataset:
            mean_std_values = dataset[method_key]
            for mean, std in mean_std_values:
                row.append(f"{mean:.2f} ({std:.2f})")
        else:
            row.extend([""] * 3)
    table_data.append(row)

# Transfer to DataFrame
df = pd.DataFrame(table_data, index=pd.MultiIndex.from_tuples(methods, names=["Objective", "Regularization", "Posterior Predictive"]), columns=columns)

# Design the table style
df_styled = df.style.set_caption("Results Table").set_table_styles([
    {"selector": "th", "props": [("text-align", "center")]},
    {"selector": "td", "props": [("text-align", "center")]}
])

# Export to LaTeX
latex_table = df.to_latex(multicolumn=True, multicolumn_format='c', index=True)

with open("results_table.tex", "w") as file:
    file.write(latex_table)

# Save the table
df.to_csv("results_table.csv")


df_styled